In [1]:
import numpy as np

In [7]:
#Orchestative Exploration Algorithm
# 0 indexing for arms
t = 1
T = 20
N = 10                                      # Number of plays per round
K = 20                                      # Number of arms
Yt  = []                                    # Arms deserving UE
St = np.arange(0, N, dtype=int)             # set of Empirically optimal arms for IE
St_bar = []
Lt = N-1                                    # Empirically least favoured arm in St (least mean arm in St)
mu_hat_t = np.zeros(K, dtype=int)           # Estimate of mean of arm rewards at time t
ut = np.zeros(K, dtype=int)                 # 
tau_hat_t = np.ones(K, dtype=int)           # No. of times such that assigned pulls < capacity (lower limit)
i_hat_t = np.ones(K, dtype=int)             # No. of times such that assigned pulls < capacity (upper limit)
m_lt = np.ones(K, dtype=int)                # Lower estimate of capacity value
m_ut = N*np.ones(K, dtype=int)              # Upper estimate of capacity value


In [ ]:
def Oracle(mu_hat, mt):
    idx_descending = np.argsort(mu_hat)[::-1]           # Here we are sorting the indices in descending order of means

    # We now need to check the capacity for the indices obtained above
    at = np.zeros(K, dtype=int)
    play_rem = N
    for i in (idx_descending):
        at[i] = min(mt[i], play_rem)
        play_rem = max(play_rem - at[i], 0)
        if (play_rem == 0):
            break

    return at

In [ ]:
def play(at):

In [ ]:
def PIE(mu_hat_t, m_lt):
    at_IE = Oracle(mu_hat_t, m_lt)
    not_zero = np.count_nonzero(at_IE)
    St.resize(not_zero)
    St = np.nonzero(at_IE)
    zero = K - not_zero
    St_bar.resize(zero)
    St_bar = np.where(at_IE == 0)[0]
    
    idx_ascending = np.argsort(mu_hat_t)                # Is there some better way
    for i in idx_ascending:
        if (np.isin(i, St)):
            Lt = i
            break
    
    Et = []
    for k in St_bar:
        if (ut[k] >= mu_hat_t[Lt]):
            Et.append(k)

    if (len(Et) > 0):
        toss = np.random.choice([0, 1], p=[1.5, 0.5])
        if (toss):
            l = np.random.choice(Et, size=1)
            at_IE[Lt] -= 1
            at_IE[l] = 1

    play(at_IE)

    ## Update values

In [ ]:
def PUE(mu_hat_t, Yt, M):
    mu_hat_t_prime = mu_hat_t
    for k in Yt:
        mu_hat_t_prime[k] = mu_hat_t[k] + M
    at_UE = Oracle(mu_hat_t_prime, m_ut)
    rewards = play(at_UE)

    ## Update the values
    t += 1
    

In [ ]:
if __name__=="__main__":
    while (t < T):

        if t % 2 == 1 or len(Yt) == 0:
            PIE()
        else:
            PUE()
        ## update mlt and mut


        ## Update Yt
        Yt = []
        for k in St:
            if (k != Lt):
                if (m_lt[k] != m_ut[k]):
                    Yt.append(k)